<a href="https://colab.research.google.com/github/dneves86/DIO-Potencia-Tech-Ifood/blob/main/Otimizando_Sistema_Banc%C3%A1rio_com_Fun%C3%A7%C3%B5es_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Desafio

Precisamos deixar nosso código mais modularizado, para isso vamos criar funções para as operações existentes: sacar, depositar e visualizar histórico. Além disso, para a versão 2 do nosso sistema precisamos criar duas novas funções: criar usuário (cliente do banco) e criar conta corrente (vincular com usuário).

In [1]:
import textwrap

def menu():
  menu = """\n
  ================= MENU =================
  [d]\tDepositar
  [s]\tSacar
  [e]\tExtrato
  [nc]\tNova conta
  [lc]\tListar contas
  [nu]\tNovo usuário
  [q]\tSair
  => """
  return input(textwrap.dedent(menu))

##Separação em funções

Devemos criar funções para todas as operações do sistema. Para exercitar tudo o que aprendemos neste módulo, cada função vai ter uma regra na passagem de argumentos.

###Saque

A função saque deve receber os argumentos apenas por nome (keyword only). Sugestão de argumentos: saldo, valor, extrato, limite, numero_saques, limite_saques. Sugestão de retorno: saldo e extrato.

In [11]:
def sacar(*, saldo, valor, extrato, limite, numero_saques, limite_saques):
  excedeu_saldo = valor > saldo
  excedeu_limite = valor > limite
  excedeu_saques = numero_saques >= limite_saques

  if excedeu_saldo:
    print('Operaçao falhou! Você não tem saldo suficiente!')

  elif excedeu_limite:
    print('Operação falhou! O valor do saque excede o limite!')

  elif excedeu_saques:
    print('Operação falhou! Número máximo de saques foi excedido!')

  elif valor > 0:
    saldo -= valor
    extrato += f'Saque:\t\tR$ {valor:.2f}\n'
    numero_saques += 1
    print('\nSaque realizado com sucesso!')

  else:
    print('Operação falho! O valor informado é inválido!')

  return saldo, extrato

###Depósito

A função depósito deve receber os argumentos apenas por posição (positional only). Sugestão de argumentos: saldo, valor, extrato. Sugestão de retorno: saldo e extrato.

In [3]:
def depositar(saldo, valor, extrato, /):

    if valor > 0:
      saldo += valor
      extrato += f'Depósito: R$ {valor:.2f}\n'
      print('\nDepósito realizado com sucesso!')

    else:
      print('\nOperação falhou! O valor informado é invalido')

    return saldo, extrato

###Extrato

A função extrato deve receber os argumentos por posição e nome (positional only e keyword only). Argumentos posicionais: saldo. Argumentos nomeados: extrato.

In [4]:
def exibir_extrato(saldo, /, *, extrato):
  print('\n================ EXTRATO ================')
  print('Não foram realizadas movimentações.' if not extrato else extrato)
  print(f'\nSaldo:\t\tR$ {saldo:.2f}')
  print('==========================================')

###Novas funções

Precisamos criar duas novas funções: criar usuário e criar conta corrente.

####Criar usuário (cliente)

O programa deve armazenar os usuários em uma lista. Um usuário é composto por: nome, data de nascimento, cpf e endereço. O endereço é uma string com formato: logradouro, nro - bairro - cidade/sigla estado. Deve ser armazenado somente os números do CPF. Não podemos cadastrar dois usuários com o mesmo CPF.

In [5]:
def criar_usuario(usuarios):
  cpf = input('Informe o CPF (somente número): ')
  usuario = filtrar_usuario(cpf, usuarios)

  if usuario:
    print(print('\nJá existe usuário com esse CPF!'))
    return

  nome = input('Informe o nome completo: ')
  data_nascimento = input('Informe a data de nascimento (dd-mm-aaaa): ')
  endereco = input('Informe o endereço (logradouro, nro - bairro - cidade/sigla estado): ')

  usuarios.append({'nome': nome, 'cpf': cpf, 'data_nascimento': data_nascimento, 'endereco': endereco})

  print('Usuário criado com sucesso!')

def filtrar_usuario(cpf, usuarios):
  usuarios_filtrados = [usuario for usuario in usuarios if usuario['cpf'] == cpf]
  return usuarios_filtrados[0] if usuarios_filtrados else None

####Criar conta corrente

O programa deve armazenar contas em uma lista, uma conta é composta por: agência, número da conta e usuário. O número da conta é sequencial, iniciando em 1. O número da agência é fixo: "0001". O usuário pode ter mais de uma conta, mas uma conta pertence a somente um usuário.

In [6]:
def criar_conta(agencia, numero_conta, usuarios):
  cpf = input('Informe o CPF do usuário: ')
  usuario = filtrar_usuario(cpf, usuarios)

  if usuario:
    print('\nConta criada com sucesso!')
    return {'agencia': agencia, 'numero_conta': numero_conta, 'usuario': usuario}

  print('\nUsuário não encontrado, fluxo de criação de conta encerrado!')

def listar_contas(contas):
  for conta in contas:
    linha = f"""\
      Agência:\t{conta['agencia']}
      C/C:\t\t{conta['numero_conta']}
      Titular:\t{conta['usuario']['nome']}
    """
    print('=' * 100)
    print(textwrap.dedent(linha))

##Main

In [12]:
def main():
  LIMITE_SAQUES = 3
  AGENCIA = '0001'

  saldo = 0
  limite = 500
  extrato = ''
  numero_saques = 0
  usuarios = []
  contas = []

  while True:
    opcao = menu()

    if opcao == 'd':
      valor = float(input('Informe o valor do depósito: '))

      saldo, extrato = depositar(saldo, valor, extrato)

    elif opcao == 's':
      valor = float(input('Informe o valor do saque: '))

      saldo, extrato = sacar(
          saldo = saldo,
          valor = valor,
          extrato = extrato,
          limite = limite,
          numero_saques = numero_saques,
          limite_saques = LIMITE_SAQUES,
      )

    elif opcao == 'e':
      exibir_extrato(saldo, extrato=extrato)

    elif opcao == 'nu':
      criar_usuario(usuarios)

    elif opcao == 'nc':
      numero_conta = len(contas) + 1
      conta = criar_conta(AGENCIA, numero_conta, usuarios)

      if conta:
        contas.append(conta)

    elif opcao == 'lc':
      listar_contas(contas)

    elif opcao =='q':
      break

    else:
      print('Operação inválida, por favor selecione novamente a operação desejada.')

In [13]:
main()



================= MENU =================
[d]	Depositar
[s]	Sacar
[e]	Extrato
[nc]	Nova conta
[lc]	Listar contas
[nu]	Novo usuário
[q]	Sair
=> s
Informe o valor do saque: 500
Operaçao falhou! Você não tem saldo suficiente!


================= MENU =================
[d]	Depositar
[s]	Sacar
[e]	Extrato
[nc]	Nova conta
[lc]	Listar contas
[nu]	Novo usuário
[q]	Sair
=> d
Informe o valor do depósito: 1000

Depósito realizado com sucesso!


================= MENU =================
[d]	Depositar
[s]	Sacar
[e]	Extrato
[nc]	Nova conta
[lc]	Listar contas
[nu]	Novo usuário
[q]	Sair
=> s
Informe o valor do saque: 500

Saque realizado com sucesso!


================= MENU =================
[d]	Depositar
[s]	Sacar
[e]	Extrato
[nc]	Nova conta
[lc]	Listar contas
[nu]	Novo usuário
[q]	Sair
=> e

================ EXTRATO ================
Depósito: R$ 1000.00
Saque:		R$ 500.00


Saldo:		R$ 500.00


================= MENU =================
[d]	Depositar
[s]	Sacar
[e]	Extrato
[nc]	Nova conta
[lc]	Listar co